# **Recomendation By Product**

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import re
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from tensorflow import keras
from keras import layers

In [2]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
dataSet = pd.read_parquet('/content/gdrive/MyDrive/dataset/train.parquet',columns =["product_id","brand", "cat_0","cat_1","cat_2","cat_3"])
dataSet = dataSet.iloc[:100000]

dataSet

,product_id,brand,cat_0,cat_1,cat_2,cat_3
0,1005014,samsung,electronics,smartphone,NA,NA
1,1005115,apple,electronics,smartphone,NA,NA
2,1002542,apple,electronics,smartphone,NA,NA
3,1002542,apple,electronics,smartphone,NA,NA
4,4804056,apple,electronics,audio,headphone,NA
...,...,...,...,...,...,...
99995,1002524,apple,electronics,smartphone,NA,NA
99996,3100959,scarlett,appliances,kitchen,blender,NA
99997,4802272,samsung,electronics,audio,headphone,NA
99998,1004249,apple,electronics,smartphone,NA,NA


In [5]:
categories = (dataSet.cat_0 + '|' + dataSet.cat_1.astype(str) + '|' + dataSet.cat_2.astype(str)  + '|' + dataSet.cat_3.astype(str) )

In [6]:
dataAll = pd.DataFrame({
    'product': dataSet['product_id'].tolist(),
    'brand': dataSet['brand'].tolist(),
    'category': categories.tolist()
})

In [7]:
dataAll = dataAll.drop_duplicates('product')

In [8]:
product = dataAll['product'].tolist()

category = dataAll['category'].tolist()

dataDetails = pd.DataFrame({
    'product': product,
     'brand': dataAll['brand'].tolist(),
    'category': category
})

In [9]:
# CountVectorizer is used to convert a given text into a vector based on the frequency (number) of each word appearing throughout the text
tf = CountVectorizer()

tf.fit(category) 

# tf.get_feature_names_out()

CountVectorizer()

In [10]:
tfidf_matrix = tf.fit_transform(category) 

# tfidf_matrix.shape
# tfidf_matrix.todense()

In [12]:
# Calculating cosine similarity on the tf-idf matrix
cosine_sim = cosine_similarity(tfidf_matrix) 
cosine_sim

array([[1.        , 1.        , 1.        , ..., 0.61237244, 0.81649658,
        0.40824829],
       [1.        , 1.        , 1.        , ..., 0.61237244, 0.81649658,
        0.40824829],
       [1.        , 1.        , 1.        , ..., 0.61237244, 0.81649658,
        0.40824829],
       ...,
       [0.61237244, 0.61237244, 0.61237244, ..., 1.        , 0.5       ,
        0.25      ],
       [0.81649658, 0.81649658, 0.81649658, ..., 0.5       , 1.        ,
        0.5       ],
       [0.40824829, 0.40824829, 0.40824829, ..., 0.25      , 0.5       ,
        1.        ]])

In [16]:
indices = pd.Series(index = dataDetails['product'], data = dataDetails.index).drop_duplicates()
indices.head()

product
1005014    0
1005115    1
1002542    2
4804056    3
1004873    4
dtype: int64

In [17]:
def get_recommendations(product, cosine_sim = cosine_sim,items=dataDetails[['product','brand','category']]):
   
 
    if product in indices :
      idx = indices[product]
    else :
      return 'No result'   
    
    
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    sim_scores = sorted(sim_scores, key = lambda x : x[1], reverse = True)
    
    sim_scores = sim_scores[1:30]
    
    product_indices = [i[0] for i in sim_scores]
    
    return pd.DataFrame(dataDetails['product'][product_indices]).merge(items)

In [18]:

get_recommendations('4802272')

,product,brand,category
0,4804295,xiaomi,electronics|audio|headphone|NA
1,4804055,apple,electronics|audio|headphone|NA
2,4803977,samsung,electronics|audio|headphone|NA
3,4802159,jbl,electronics|audio|headphone|NA
4,4803976,samsung,electronics|audio|headphone|NA
5,4802205,marshall,electronics|audio|headphone|NA
6,4803924,defender,electronics|audio|headphone|NA
7,4804177,xiaomi,electronics|audio|headphone|NA
8,4802930,elari,electronics|audio|headphone|NA
9,4801946,apple,electronics|audio|headphone|NA


In [ ]:
TP = 29 #number of correct predictions for similar or similar categories
FP = 0 #number of incorrect predictions for similar or similar categories

precision = TP/(TP+FP)
print("{0:.0%}".format(precision))

# **Recommendation By User**

In [ ]:
# install the tensorflow recommender library
!pip install -q tensorflow-recommenders

In [ ]:
# before running this cell download kaggle token from your account on kaggle then upload the file here
! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

# Make directory named kaggle and copy kaggle.json file there.
! chmod 600 ~/.kaggle/kaggle.json

# Change the permissions of the file.
! kaggle datasets download -d dschettler8845/recsys-2020-ecommerce-dataset

 99% 621M/629M [00:20<00:00, 45.0MB/s]
100% 629M/629M [00:20<00:00, 32.3MB/s]


In [ ]:
! unzip /content/recsys-2020-ecommerce-dataset.zip

Archive:  /content/recsys-2020-ecommerce-dataset.zip
  inflating: test.parquet            
  inflating: train.parquet           
  inflating: val.parquet             


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn import model_selection
import tensorflow_recommenders as tfrs

In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [ ]:
dataset = pd.read_parquet('/content/train.parquet')
# test_dataset = pd.read_parquet('/content/test.parquet')
# val_dataset = pd.read_parquet('/content/val.parquet')


In [ ]:
# dataset = pd.concat([train_dataset, test_dataset, val_dataset], ignore_index=True)
# del train_dataset
# del test_dataset
# del val_dataset

In [ ]:
dataset

In [ ]:
data = dataset[['user_id', 'product_id', 'brand',	'price', 'cat_0', 'cat_1', 'cat_2', 'cat_3']]
data

In [ ]:
data.price = data.price.astype(np.float32)

data.price.hist

In [ ]:
# data.replace('NA', np.nan, inplace=True)


In [ ]:

# data.dropna(inplace=True)
# data.reset_index(drop=True, inplace=True)


In [ ]:
data

In [ ]:
# create function to get the unique values of a column with there index
def create_unique_to_id(data):
    unique_id = {}
    id_unique = {}
    for i, x in enumerate(set(data)):
        # make sure to add 1 to the index to avoid 0
        unique_id[x] = i + 1
        id_unique[i + 1] = x
    return unique_id, id_unique

In [ ]:
id_product, product_id = create_unique_to_id(data.product_id)
id_user, user_id = create_unique_to_id(data.user_id)
id_brand, brand_id = create_unique_to_id(data.brand)
# id_cat0, cat0_id = create_unique_to_id(data.cat_0)
# id_cat1, cat1_id = create_unique_to_id(data.cat_1)
# id_cat2, cat2_id = create_unique_to_id(data.cat_2)


In [ ]:
users_ids = data.user_id.replace(user_id)
products_ids = data.product_id.replace(product_id)
# cat0 = data.cat_0.replace(cat0_id)
# cat1 = data.cat_1.replace(cat1_id)
# cat2 = data.cat_2.replace(cat2_id)


In [ ]:
# create the function of data pipeline
def create_datapipelline(x1=None, x2=None, batch_size=64):
    # if we have two inputs
    if type(x2) != type(None):
        data = tf.data.Dataset.from_tensor_slices(((x1, x2)))
        data = data.shuffle(1000000)
        data = data.batch(batch_size)
    # if we have one input
    # note we don't need to shuffle the data for the candidate movies
    else:
        data = tf.data.Dataset.from_tensor_slices((x1))
        data = data.batch(batch_size)

    data = data.prefetch(tf.data.AUTOTUNE)
    # return the data pipeline
    return data


# create the data pipeline for the input data with the batch size of 8192
inputs_pipeline = create_datapipelline(users_ids, products_ids, batch_size=8192)
# create the data pipeline for the candidate products with the batch size of 128
products_candidates_pipeline = create_datapipelline(products_candidates, batch_size=128)

In [ ]:
# get the length of the user ids
users_unique_length = len(set(users_ids))
# create the user id model
# the model will contain one embedding layer with the size of the user id
class UserModel(tf.keras.Model):
    def __init__(self, users_unique_length, **kargs):
        super(UserModel, self).__init__(**kargs)
        # user embedding layer
        self.embedding = tf.keras.layers.Embedding(
            users_unique_length + 1, 64, input_length=1, mask_zero=True
        )

    def call(self, x):
        x = self.embedding(x)
        return x


# get the length of the product ids
products_unique_length = len(set(products_ids))
# create the product id model
# the model will contain one embedding layer with the size of the product id
class ProductModel(tf.keras.Model):
    def __init__(self, products_unique_length, **kargs):
        super(MovieModel, self).__init__(**kargs)
        # product embedding layer
        self.embedding = tf.keras.layers.Embedding(
            products_unique_length + 1, 64, input_length=1, mask_zero=True
        )

    def call(self, x):
        x = self.embedding(x)
        return x

In [ ]:
# create the user model & product model
product_model = ProductModel(products_unique_length, name="Products_Model")

user_model = UserModel(users_unique_length, name="Users_Model")

In [ ]:
# metrics
metrics = tfrs.metrics.FactorizedTopK(
    candidates=product_candidates_pipeline.map(product_model)
)

In [ ]:
# loss function
task = tfrs.tasks.Retrieval(metrics=metrics)

In [ ]:
class ProductlensModel(tfrs.Model):
    def __init__(self, user_model, product_model, task):
        super().__init__()

        self.product_model = product_model
        self.user_model = user_model
        self.task = task

    def compute_loss(self, features, training=False):
        user_ids = features[0]
        product_ids = features[1]
        # We pick out the user features and pass them into the user model.
        user_embeddings = self.user_model(user_ids)
        # And pick out the product features and pass them into the product model,
        # getting embeddings back.
        positive_product_embeddings = self.product_model(product_ids)

        # The task computes the loss and the metrics.
        return self.task(user_embeddings, positive_product_embeddings)

In [ ]:
# create the productlens model
model = ProductlensModel(user_model, product_model, task)
# compile the model with the adam optimizer with learning rate of 0.1
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.1))

In [ ]:
# train the model with 50 epochs
model.fit(inputs_pipeline, epochs=50)

# **Model Deployment**

In [ ]:

def recommend(product, cosine_sim = cosine_sim, items = dataDetails[['product','brand','category']]):
     
    try:
        if product in indices :
            idx = indices[product]
        else :
            return 'No result'   
        
        
        sim_scores = list(enumerate(cosine_sim[idx]))
        
        sim_scores = sorted(sim_scores, key = lambda x : x[1], reverse = True)
        
        sim_scores = sim_scores[1:30]
        
        product_indices = [i[0] for i in sim_scores]
        
        return pd.DataFrame(dataDetails['product'][product_indices]).merge(items)
    except:
        return 'No result'

In [ ]:
! pip install -q gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.2/14.2 MB 59.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 KB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 KB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 57.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 KB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 KB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 KB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 KB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.6/80.6 KB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 KB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import gradio as gr
iface = gr.Interface(recommend, 
             'text', 
             'dataframe', theme='dark-peach', title='E-commerce Recomender', 
             description='Here you will see the recomendation of each product',
             )
iface.launch(debug=True)

ModuleNotFoundError: ignored